# Data Preparation for Feature-Based Cluster Queries

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
chrom = 'chr7'
bin_size = 100000

chrom_sizes_file = '../data/hg19/hg19.chrom.sizes'

chromhmm_5kb_bed_file = '../data/carlo/HG19_chromhmm_5.0Kb_allchr.bed.gz'

ffr_motif_bed_file = '../data/carlo/FFR_constels_0607_50k-1M.BED'
frf_motif_bed_file = '../data/carlo/FRF_constels_0607_50k-1M.BED'
frr_motif_bed_file = '../data/carlo/FRR_constels_0607_50k-1M.BED'
rfr_motif_bed_file = '../data/carlo/RFR_constels_0607_50k-1M.BED'

In [4]:
from hgmc.utils import get_chrom_sizes

chrom_size = get_chrom_sizes(chrom_sizes_file).get(chrom)
num_bins = math.ceil(chrom_size / bin_size)

# Convert Data

### Convert BED to SQL

In [6]:
from hgmc.bed import bed_to_sql
    
bed_to_sql(chromhmm_5kb_bed_file, chrom_sizes_file)

In [6]:
from hgmc.bed import bed_to_sql

bed_to_sql(ffr_motif_bed_file, chrom_sizes_file)
bed_to_sql(frf_motif_bed_file, chrom_sizes_file)
bed_to_sql(frr_motif_bed_file, chrom_sizes_file)
bed_to_sql(rfr_motif_bed_file, chrom_sizes_file)

ValueError: Either provide a BED4 file or provide `feature_name`

In [18]:
columns = [
    'chrom',
    'start',
    'end',
    'score',
    'unknown1',
    'strand',
    'thickStart',
    'thickEnd',
    'itemRgb',
    'unknown4',
    'unknown5',
    'unknown6'
]

In [21]:
import pandas as pd

ffr = pd.read_table(ffr_motif_bed_file, sep=' ', header=None)
ffr.columns = columns
ffr['name'] = 'ffr'
frf = pd.read_table(frf_motif_bed_file, sep=' ', header=None)
frf.columns = columns
frf['name'] = 'frf'
frr = pd.read_table(frr_motif_bed_file, sep=' ', header=None)
frr.columns = columns
frr['name'] = 'frr'
rfr = pd.read_table(rfr_motif_bed_file, sep=' ', header=None)
rfr.columns = columns
rfr['name'] = 'rfr'

In [22]:
combined_motifs = pd.concat([
    ffr[['chrom', 'start', 'end', 'name']],
    frf[['chrom', 'start', 'end', 'name']],
    frr[['chrom', 'start', 'end', 'name']],
    rfr[['chrom', 'start', 'end', 'name']]
], ignore_index=True)

In [27]:
all_motifs_bed_file = '../data/carlo/all_constels_0607_50k-1M.bed'
combined_motifs.to_csv(all_motifs_bed_file, sep='\t', index=False, header=False)

In [28]:
bed_to_sql(all_motifs_bed_file, chrom_sizes_file)

---

In [30]:
chromhmm_5kb = pd.read_table(chromhmm_5kb_bed_file, header=None)
chromhmm_5kb.head()

,0,1,2,3
0,chr1,0,4999,-
1,chr1,5000,9999,-
2,chr1,10000,14999,15_Repetitive/CNV+13_Heterochrom/lo+8_Insulato...
3,chr1,15000,19999,10_Txn_Elongation
4,chr1,20000,24999,10_Txn_Elongation+11_Weak_Txn+9_Txn_Transition...
